In [1]:
# run.py

!pip install transformers
!pip install faiss-cpu
!pip install sentence-transformers
!pip install torch
!pip install gradio
!pip install vllm

In [2]:
from retriever import initialize_retriever, retrieve_top_chunks
from generator import initialize_generator
from generator import generate_response
import logging

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Paths to data
faiss_path = "/content/100_chunk_faiss_index.bin"
metadata_path = "/content/100_chunk_metadata.json"

# Parameter dictionaries for retrieval and generation
retriever_params = {
    "initial_top_k": 30,
    "final_top_k": 2,
}

generator_params = {
    "sampling_params": {
        "temperature": 0.3,
        "top_p": 0.94,
        "max_tokens": 150,
        "repetition_penalty": 1.4,
        "top_k": 30
    }
}

In [4]:
def setup():
    """Initialize retrieval and generator components."""
    index, metadata, retriever_model = initialize_retriever(faiss_path, metadata_path)
    model, tokenizer, streamer, sampling_params = initialize_generator()
    generator_params["sampling_params"] = sampling_params
    logging.info("Setup complete.")

    return index, metadata, retriever_model, model, tokenizer, streamer


# RUN

In [5]:
# Initialize models and data only once
index, metadata, retriever_model, model, tokenizer, streamer = setup()

FAISS index loaded successfully.
Metadata loaded successfully.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Retriever model initialized successfully.
INFO 11-05 09:12:35 awq_marlin.py:101] Detected that the model can run with awq_marlin, however you specified quantization=awq explicitly, so forcing awq. Use quantization=awq_marlin for faster inference
WARNING 11-05 09:12:35 config.py:321] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
WARNING 11-05 09:12:35 config.py:395] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 11-05 09:12:35 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='speakleash/Bielik-7B-Instruct-v0.1-AWQ', speculative_config=None, tokenizer='speakleash/Bielik-7B-Instruct-v0.1-AWQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1500, download_dir

/usr/local/lib/python3.10/dist-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/usr/local/lib/python3.10/dist-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")


INFO 11-05 09:12:37 model_runner.py:1056] Starting to load model speakleash/Bielik-7B-Instruct-v0.1-AWQ...
INFO 11-05 09:12:37 selector.py:247] Cannot use FlashAttention-2 backend due to sliding window.
INFO 11-05 09:12:37 selector.py:115] Using XFormers backend.
INFO 11-05 09:12:37 weight_utils.py:243] Using model weights format ['*.safetensors']
INFO 11-05 09:12:38 weight_utils.py:288] No model.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 11-05 09:12:39 model_runner.py:1067] Loading model weights took 3.8745 GB
INFO 11-05 09:12:41 gpu_executor.py:122] # GPU blocks: 3437, # CPU blocks: 2048
INFO 11-05 09:12:41 gpu_executor.py:126] Maximum concurrency for 1500 tokens per request: 36.66x


In [6]:
# Start the chatbot interface
#gradio_interface(index, metadata, retriever_model, model, tokenizer, streamer, retriever_params, generator_params)

In [7]:
# Example query
query = "co jest ważne dla pracownika?"

# Step 1: Retrieve context based on the query
retrieved_chunks = retrieve_top_chunks(query, index, metadata, retriever_model, **retriever_params)

# Combine retrieved chunks into a single context string for input to the generator
context = "\n".join(f"{chunk['speaker']}:\n{chunk['text']}" for chunk in retrieved_chunks)
# Print the context to confirm it includes all 3 chunks
print("Final Context Passed to Generator:\n", context)

# Step 2: Generate response based on the retrieved context with a single display of output
response = generate_response(
    query=query,
    context=context,
    model=model,
    tokenizer=tokenizer,
    streamer=streamer,
    sampling_params=generator_params["sampling_params"]
)

# Print the final generated response only once
print("Generated Response:\n", response)

Filtered chunks count (should be 2): 2
Final Context Passed to Generator:
 Grzegorz Rutkowski:
kogoś kto traktuje pracowników jako jeden z asetów. mam taki zasób w postaci pracowników, oni dodam są nie zaangażowani, nie chce im się interesuje ich tylko pensja i tak naprawdę nie chcą brać odpowiedzialności za wynik. a to nie jest prawda. pracownicy zawsze chcą brać odpowiedzialność za wynik, albo inaczej chcą wiedzieć, jakie są wyniki. i to jest absolutnie baza. i w momencie, kiedy my rozmawiamy z pracownikami, komunikujemy się z nimi, przedstawiamy, jak jest kierunek do konzymiarza firma, po co my to robimy, jakie są cele. i to w tym czysto wydaniu technicznym od tam budżetu, w przeskadę celów,
Grzegorz Pietrusiński:
na to jest dla niego to dla to okierownika, bo to jest to, że jest, mógł ugolnić część swojego odnia pracy, na inny zadania, które też były ważne. pan grzegorz ludkowski,


Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.84s/it, est. speed input: 63.06 toks/s, output: 43.87 toks/s]

Generated Response:
 Grzegorz Rutkowski:
to, co jest ważne dla pracownika, to jest to, że on chce wiedzieć, co się dzieje w firmie. i to jest bardzo ważne. i to jest bardzo ważne. i to jest bardzo ważne. i to jest bardzo ważne. i to jest bardzo ważne. i to jest bardzo ważne. i to jest bardzo ważne. i to jest bardzo ważne. i to jest bardzo ważne. i to jest bardzo ważne. i to jest bardzo ważne. i to jest bardzo ważne. i to jest bardzo ważne. i to jest bardzo ważne. i to jest bardzo ważne. i to jest bardzo ważne. i to jest bardzo ważne. i to jest bardzo ważne. i to jest bardzo ważne. i to jest bardzo ważne. i to jest bardzo ważne.
